Import all variables/ libraries from main

In [2]:
from main import data, pd, plt, train_test_split, GaussianNB, accuracy_score, DecisionTreeClassifier, export_text, plot_tree, sns, apriori, association_rules, data_x, target
import constants
import utils
import pprint

pp = pprint.PrettyPrinter(indent=4)

2022-12-27 20:26:04.552043: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-27 20:26:04.552209: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-27 20:26:04.552226: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


Print data

In [3]:
data

,Systemic_Illness,Rectal_Pain,Sore_Throat,Penile_Oedema,Oral_Lesions,Solitary_Lesion,Swollen_Tonsils,HIV_Infection,STI,MonkeyPox
Patient_ID,,,,,,,,,,
P0,None,False,True,True,True,False,True,False,False,Negative
P1,Fever,True,False,True,True,False,False,True,False,Positive
P2,Fever,False,True,True,False,False,False,True,False,Positive
P3,None,True,False,False,False,True,True,True,False,Positive
P4,Swollen Lymph Nodes,True,True,True,False,False,True,True,False,Positive
...,...,...,...,...,...,...,...,...,...,...
P24995,None,True,True,False,True,True,False,False,True,Positive
P24996,Fever,False,True,True,False,True,True,True,True,Positive
P24997,None,True,True,False,False,True,True,False,False,Positive


Plot data, shows how often MonkeyPox is negative and positive

In [13]:
series = data[constants.MONKEY_POX].value_counts()

# TODO: Try to use countplot and change colors to normal blue and orange
plt.bar(series.index, series.values, color=constants.COLOR_PALETTE)
plt.title('Cases of Monkey Pox seperated by positive and negative')
plt.show()

True     12527
False    12473
Name: Solitary_Lesion, dtype: int64

In [ ]:
df_corr  = data.corr()
df_corr

Plots heatmap: Shows correlations between the different features

In [ ]:
sns.heatmap(df_corr, annot=True, vmin=0.5, vmax=0.7)
plt.show()

Shows how often each systemic illness is causing MonkeyPox to be positive or negative

In [14]:
# lift = 1
# confidence = 1 is good?
symptoms_df = utils.clean_data(data)
frequency = apriori(symptoms_df, min_support=0.005, use_colnames=True)
rules = association_rules(frequency, metric='lift')
rules.sort_values('confidence', ascending=False, inplace=True)
rules

/home/linus/Daten_Linus/Studium/IW7/Data_Mining/Project_Data_Mining/utils.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_concat[col][index] = True


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
20254,"(Penile_Oedema, None, Oral_Lesions, Swollen_To...",(Rectal_Pain),0.015086,0.546986,0.009114,0.604167,1.104538,0.000863,1.144456
36017,"(Rectal_Pain, HIV_Infection, Swollen Lymph Nod...",(Sore_Throat),0.010246,0.526117,0.006160,0.601227,1.142762,0.000770,1.188352
19697,"(None, HIV_Infection, Oral_Lesions, Sore_Throa...",(Rectal_Pain),0.014646,0.546986,0.008737,0.596567,1.090643,0.000726,1.122896
31114,"(Penile_Oedema, Rectal_Pain, Fever, Oral_Lesio...",(Solitary_Lesion),0.010874,0.515243,0.006474,0.595376,1.155524,0.000871,1.198042
20006,"(Penile_Oedema, Solitary_Lesion, None, Oral_Le...",(Rectal_Pain),0.013452,0.546986,0.007983,0.593458,1.084960,0.000625,1.114310
...,...,...,...,...,...,...,...,...,...
37898,(Rectal_Pain),"(Penile_Oedema, HIV_Infection, Muscle Aches an...",0.546986,0.009491,0.005029,0.009193,0.968583,-0.000163,0.999699
37773,(HIV_Infection),"(Penile_Oedema, Rectal_Pain, Muscle Aches and ...",0.558615,0.009303,0.005091,0.009114,0.979740,-0.000105,0.999810
34375,(HIV_Infection),"(Penile_Oedema, Rectal_Pain, None, Solitary_Le...",0.558615,0.009240,0.005029,0.009002,0.974227,-0.000133,0.999760
36639,(HIV_Infection),"(Penile_Oedema, Rectal_Pain, Swollen Lymph Nod...",0.558615,0.010120,0.005029,0.009002,0.889512,-0.000625,0.998872


In [16]:
# TODO: Change legend and axises
sns.countplot(x=data[constants.SYSTEMIC_ILLNESS], hue=data[constants.MONKEY_POX], palette=constants.COLOR_PALETTE)
plt.show()

Plots data, using pairplot

In [ ]:
#sns.pairplot(data.head(100), hue='MonkeyPox')

Calculates accuracy, using GaussianNB()

In [ ]:
ran_stream = 23
x_train, x_test, y_train, y_test = train_test_split(data_x, target, random_state=ran_stream)

model = GaussianNB()
model.fit(x_train, y_train)
y_prediction = model.predict(x_test)
print(100 * accuracy_score(y_test, y_prediction))

Creates DecisionTree, by using entropy

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_x, target, test_size=0.30)
model_tree = DecisionTreeClassifier(criterion='entropy', splitter='best', min_samples_split=0.10)
model_tree.fit(x_train, y_train)
# Accuracy checken.
y_prediction = model_tree.predict(x_test)
#print(100 * accuracy_score(y_test, y_prediction))
text_representation = export_text(model_tree, feature_names=x_train.columns.values.tolist())
figure = plt.figure(figsize=(10,8))
plot_tree(model_tree, feature_names=x_train.columns, filled=True, rounded=True)
plt.show()

Plots data, when symptom (2-7) is True and MonkeyPox is Positive

In [ ]:
symptoms = data.iloc[:, 1:-1].columns.values.tolist()
symptoms_df = data.iloc[:, 1:]
symptoms_df = utils.fetch_true_symptoms(symptoms, symptoms_df)
print(symptoms_df)

pos_lst, negative_lst = utils.create_outcome_lists(symptoms, symptoms_df)
df1 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': pos_lst})
df2 = pd.DataFrame({'Symptoms': symptoms, 'Number of infections': negative_lst})
df1['outcome'] = 'Positive'
df2['outcome'] = 'Negative'
res = pd.concat([df1, df2])
sns.barplot(x='Symptoms', y='Number of infections', data=res, hue='outcome', palette=constants.COLOR_PALETTE)
plt.show()

Calculates accuracy by removing different symptoms, checks if accuracy is higher or lower depending on which symptom is removed

In [ ]:
temp_dict = utils.create_feature_accuracy_dict(data_x, target)
top_items = sorted(temp_dict.items(), key = lambda x: x[1], reverse=True)[:5]
pp.pprint(top_items)

Tries to detect overfitting by plotting the test and train data

In [ ]:
utils.check_over_fitting(data_x, target)